## To Do

- Finish transfer counts variable (after recoding suspect transfers)
- Get rid of double admits w/ differing service codes
- Rename all column variables to lower case
- Expand out the final patient data frame with visit ids & rows per each day in the hospital

Shape of final table:

| RUID | Visit_ID | Admit_Date | Discharge_Date | Hospital_Day | N_Transfers | Stay_Length | Readmit_Time | 30d_Readmit |
|------|----------|------------|----------------|--------------|-------------|-------------|--------------|-------------|
| user id | hospital stay # | date admitted | date discharged | date in hospital | number of transfers | duration of stay | time from last discharge to this admission | was the patient a 30d readmit? |


## Nice-to-do

- Construct missing discharge/admit dates from CPT codes (as above) -- do not do this for events where both are missing as these may be ER visits w/o admit, but do check if they fall in the range of an existing stay
- Characterize the amount of missingness of entire hospital visits from CPT codes

## Loading data

In [1]:
%ls ../data

FONNESBECK_ADT_20151202.csv        FONNESBECK_ICD9_20151202.csv*
FONNESBECK_BMI_20151202.csv*       FONNESBECK_LAB_20151202.csv*
FONNESBECK_BP_20151202.csv*        FONNESBECK_MED_20151202.csv*
FONNESBECK_CPT_20151202.csv*       FONNESBECK_phenotype_20151202.csv*
FONNESBECK_EGFR_20151202.csv*      Fonnesbeck_DD_2014102014.xlsx


In [27]:
import pandas as pd
import datetime
import numpy as np

In [2]:
adt = pd.read_table('../data/FONNESBECK_ADT_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['Admission_date','Event_Date','DISCHARGE_DATE'])
pheno = pd.read_table('../data/FONNESBECK_phenotype_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['DOB','DOD'])
cpt = pd.read_table('../data/FONNESBECK_CPT_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['Event_date'])

In [3]:
svc = pd.read_excel('../data/FONNESBECK_DD_2014102014.xlsx',sheet_name='Service code', sep = ',')
svc.rename(columns = {"Service Code":"SVC", "Service Code Desc":"Desc"}, inplace = True)

In [4]:
adt.Event = pd.Categorical(adt.Event,categories = ['Admit','Transfer','Discharge'])
adt = adt.sort_values(by = ['RUID','Admission_date','Event','Event_Date']).reset_index(drop = True)
adt.head()

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
0,50135262,Admit,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12
1,50135262,Transfer,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12
2,50135262,Transfer,2007-02-08,2007-02-09,ORT,R FEMUR FX,2007-02-12
3,50135262,Discharge,2007-02-08,2007-02-12,ORT,R FEMUR FX,2007-02-12
4,50135262,Admit,2007-08-03,2007-08-03,CAR,CP,2007-08-06


In [5]:
adt.describe(include='all')

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
count,1.215300e+05,121530,119969,121530,121530,120603,119472
unique,NaN,3,4192,4279,73,13118,4195
top,NaN,Transfer,2013-03-14 00:00:00,2013-12-28 00:00:00,GMD,296.90,2010-12-23 00:00:00
freq,NaN,61636,111,69,13062,2394,111
first,NaN,NaN,2004-01-28 00:00:00,2004-01-28 00:00:00,NaN,NaN,2004-02-11 00:00:00
last,NaN,NaN,2015-11-26 00:00:00,2015-11-26 00:00:00,NaN,NaN,2015-11-23 00:00:00
mean,5.366861e+07,NaN,NaN,NaN,NaN,NaN,NaN
std,4.628206e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,5.013526e+07,NaN,NaN,NaN,NaN,NaN,NaN
25%,5.372980e+07,NaN,NaN,NaN,NaN,NaN,NaN


## Looking at missingness

In [6]:
adt.isnull().sum()/adt.shape[0]

RUID               0.000000
Event              0.000000
Admission_date     0.012845
Event_Date         0.000000
SRV_CODE           0.000000
CHIEF_COMPLAINT    0.007628
DISCHARGE_DATE     0.016934
dtype: float64

In [268]:
adt[adt.Admission_date.isnull() & adt.DISCHARGE_DATE.isnull()].SRV_CODE.value_counts() # may correspond to ER visits without admission which we don't need to predict

CAR    295
GMD    269
PED    110
NEP     69
GER     55
PUL     51
GNS     46
OBS     45
ORT     43
INF     40
NES     38
ONC     37
PGS     37
HEM     34
PON     34
URO     32
NEU     31
GAS     23
EMR     22
TRA     20
OTO     20
HEP     17
PLS     16
EGS     15
LTS     14
PCC     14
PGA     14
PNP     11
BRN     10
PCA     10
PPU      9
PNE      9
CSX      8
RTS      8
VAS      7
PSY      6
PEN      5
GIL      4
OES      4
GYN      4
DIA      3
CLP      3
GEN      2
THS      2
NEO      2
PTA      2
ADO      2
RAD      2
CTS      2
RHM      1
Name: SRV_CODE, dtype: int64

In [266]:
adt[~(adt.Admission_date.isnull() & adt.DISCHARGE_DATE.isnull())].RUID.value_counts() # may correspond to ER visits without admission which we don't need to predict

53736286    372
53730141    366
53734944    308
53728578    288
53728001    282
53733747    258
53732256    244
53729476    230
53735136    229
53734271    213
53733351    211
53728778    204
53734307    202
53730096    198
50135437    186
53729313    185
53729615    183
53730488    181
53732315    174
53728340    173
53729068    171
53732869    170
53730727    169
53728652    168
53734565    166
53732346    161
53730100    160
53728903    158
53734444    156
53728993    155
           ... 
53729940      2
53729333      2
53731726      2
53734715      2
53732862      2
53731525      2
53732820      2
53733178      2
53736116      2
53729845      2
53736347      2
53728159      2
53728073      2
53735560      2
53732199      2
53732195      2
53735295      2
53733577      2
53733318      2
53719683      2
53728013      2
53734595      2
53733451      2
53730759      2
53733285      2
53735332      2
53728118      1
53725969      1
53734229      1
53730935      1
Name: RUID, Length: 7915

In [9]:
adt[adt.Admission_date.isnull()].Event.value_counts()

Transfer     1559
Discharge       2
Admit           0
Name: Event, dtype: int64

In [10]:
adt[adt.DISCHARGE_DATE.isnull()].Event.value_counts()

Transfer     1860
Admit         198
Discharge       0
Name: Event, dtype: int64

In [11]:
adt[(adt.Admission_date.isnull()) & (adt.Event == 'Discharge')]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
76409,53733158,Discharge,NaT,2007-08-06,TRA,STAT,2007-08-06
76578,53733172,Discharge,NaT,2013-02-17,PED,SEPSIS,2013-02-17


In [12]:
adt[76400:76420]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
76400,53733157,Admit,2012-02-14,2012-02-14,PUL,HEMOPTYSIS,2012-02-15
76401,53733157,Transfer,2012-02-14,2012-02-14,PUL,HEMOPTYSIS,2012-02-15
76402,53733157,Discharge,2012-02-14,2012-02-15,PUL,HEMOPTYSIS,2012-02-15
76403,53733157,Admit,2012-04-14,2012-04-14,ONC,FAILURE TO THRIVE; DEHYDRATION; KIDNEY CA,2012-04-15
76404,53733157,Transfer,2012-04-14,2012-04-14,ONC,FAILURE TO THRIVE; DEHYDRATION; KIDNEY CA,2012-04-15
76405,53733157,Discharge,2012-04-14,2012-04-15,ONC,FAILURE TO THRIVE; DEHYDRATION; KIDNEY CA,2012-04-15
76406,53733157,Transfer,NaT,2011-11-21,ONC,HEMOPTYSIS,NaT
76407,53733157,Transfer,NaT,2011-11-21,HEM,HEMOPTYSIS,NaT
76408,53733158,Transfer,NaT,2007-08-05,TRA,STAT,2007-08-06
76409,53733158,Discharge,NaT,2007-08-06,TRA,STAT,2007-08-06


## Adding age data & removing pediatric patients

In [14]:
adt_age = pd.merge(adt,pheno)
# admits = adt_age.Admission_date.dt
events = adt_age.Event_Date.dt
birthdays = adt_age.DOB.dt

adt_age['age'] = events.year - birthdays.year + ((events.month < birthdays.month) & (events.day < birthdays.day))
# above from https://stackoverflow.com/questions/2217488/age-from-birthdate-in-python/9754466#9754466

In [15]:
# getting rid of peds & psychiatric patients...
# we're removing these because they aren't part of the CMS criteria so 30-day readmits for them don't lose the hospital money
ped_svc = '|'.join(svc.SVC[svc.Desc.str.contains("CHILD|PED")])
psych_svc = '|'.join(svc.SVC[svc.Desc.str.contains("PSYCH")])

# ped_filter = ((adt_age.age < 18) | (adt_age.SRV_CODE.str.contains(ped_svc)) & ~((adt_age.age > 35) & (adt_age.SRV_CODE.str.contains(ped_svc))))
# the ~ condition here contains a handful of rows that I think are coding errors -- very old patients admitted to pediatric services
# the cutoff is 35 because some pediatric cancer/cardiac/etc patients will continue with pediatric services for their original condition into adulthood

ped_filter = (adt_age.age < 18)
psych_filter = (adt_age.SRV_CODE.str.contains(psych_svc))

# doing this the simplest way possible
# the psych filter will remove patients with a primary psych admit, which removes them from our consideration, but NOT patients who have psych consults
# may want to explicitly do that 

In [16]:
adt_cms = adt[~(ped_filter | psych_filter)]

In [272]:
adt_cms.RUID.value_counts()

53736286    374
53730141    363
53734944    310
53728578    289
53728001    288
53733747    251
53732256    241
53735136    229
53734271    213
53733351    211
53728778    203
53734307    201
53730096    197
53729313    185
53732315    179
53732869    173
53732346    172
53729068    172
53734565    171
53730727    171
53728652    170
50135437    158
53734444    156
53729615    156
53729476    154
53728484    152
53729822    151
53728993    148
53730650    138
53735152    138
           ... 
53732205      1
53731426      1
53728118      1
53730326      1
53730090      1
53730359      1
53730455      1
53731825      1
53730935      1
53734229      1
53727900      1
53729049      1
53734455      1
53732402      1
53734873      1
53731186      1
53729118      1
53733934      1
53734585      1
53728287      1
53734947      1
53731709      1
53729274      1
53736078      1
53728853      1
53732870      1
53732755      1
53728405      1
53730338      1
53732390      1
Name: RUID, Length: 5757

## Filtering to admits & eliminating missing discharges

In [269]:
adt_cms_admits = adt_cms[(adt_cms.Event == 'Admit') & ~(adt_cms.DISCHARGE_DATE.isnull())].copy()
# removing missing discharge dates because I can't fix them right now
# adt_cms[(adt_cms.Event == 'Admit') & (adt_cms.DISCHARGE_DATE.isnull())]

# adt_cms_admits = adt_cms_admits[adt_cms_admits.Admission_date == adt_cms_admits.Event_Date].reset_index(drop = True)
# this removes admits that aren't the same day as the admit date
# i'm not sure what these actually are--they might be miscoded transfers or admissions to another department

## Constructing variables

In [270]:
adt_cms_admits['Stay_length'] = adt_cms_admits.DISCHARGE_DATE - adt_cms_admits.Admission_date
adt_cms_admits['Readmit_time'] = adt_cms_admits.Admission_date - adt_cms_admits.DISCHARGE_DATE.shift()

didx = ~(adt_cms_admits.RUID.shift() == adt_cms_admits.RUID)

adt_cms_admits['Readmit_time'] = adt_cms_admits['Readmit_time'].mask(didx)

adt_cms_admits['30d_readmit'] = np.where(adt_cms_admits.Readmit_time <= datetime.timedelta(days=30),1,0)

In [271]:
adt_cms_admits[(adt_cms_admits.Readmit_time < datetime.timedelta(days=0)) | (adt_cms_admits.Readmit_time.shift(-1) < datetime.timedelta(days=0))] # this appears to happen when we've got two admit events for the same date with the same discharge
# in some cases they have different event dates, in other cases they're identical but have different srv_codes
# the first type might be miscoded transfers; the second type probably needs to be squished together
# ANSWERED: recode suspect transfers as Transfer but add column for "Imputed Transfer" to flag them so we can either use or not use them for Transfer counts

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,Stay_length,Readmit_time,30d_readmit
460,50135624,Admit,2015-06-24,2015-06-24,CAR,SOB,2015-06-28,4 days,94 days,0
461,50135624,Admit,2015-06-24,2015-06-25,CAR,SOB,2015-06-28,4 days,-4 days,1
528,50135821,Admit,2013-10-24,2013-10-24,GMD,LEG LAC,2013-10-29,5 days,6 days,1
529,50135821,Admit,2013-10-24,2013-10-27,GMD,LEG LAC,2013-10-29,5 days,-5 days,1
1573,50139667,Admit,2014-10-02,2014-10-02,CAR,CHF EXACERBATION,2014-10-22,20 days,900 days,0
1574,50139667,Admit,2014-10-02,2014-10-03,CAR,CHF EXACERBATION,2014-10-22,20 days,-20 days,1
1768,50141958,Admit,2014-03-11,2014-03-11,PUL,TRAUMA,2014-03-13,2 days,NaT,0
1769,50141958,Admit,2014-03-11,2014-03-12,PUL,TRAUMA,2014-03-13,2 days,-2 days,1
1804,50142794,Admit,2014-08-11,2014-08-11,CAR,CHF,2014-08-18,7 days,12 days,1
1805,50142794,Admit,2014-08-11,2014-08-12,CAR,CHF,2014-08-18,7 days,-7 days,1


In [264]:
event_counts = adt_cms[~(adt_cms.DISCHARGE_DATE.isnull())].groupby(by=['RUID','Admission_date']).Event.value_counts(sort=False).unstack(fill_value = 0)
event_counts['Transfer'] # now pull the number of transfers and we're good

RUID      Admission_date
50135262  2007-02-08        2
          2007-08-03        3
          2007-08-28        1
          2008-02-24        2
          2008-04-12        1
          2010-10-28        1
          2011-02-11        4
          2012-05-23        1
          2012-09-15        1
          2014-11-15        2
50135361  2007-12-16        3
          2008-08-02        1
          2008-10-20        1
          2008-10-30        1
          2008-11-04        2
          2008-12-02        3
          2009-03-03        3
          2009-03-29        3
          2011-10-10        1
          2011-10-20        1
          2012-01-08        0
          2012-06-09        1
          2012-06-17        1
          2012-07-27        6
50135369  2004-11-24        0
          2005-04-21        1
          2006-01-27        1
          2006-02-20        1
          2008-08-23        2
          2008-10-09        2
                           ..
53736410  2005-10-09        1
          2007-

In [260]:
adt_cms_admits[~(adt_cms_admits.Readmit_time < datetime.timedelta(days = 0))].shape

(21123, 10)

In [211]:
adt_cms.groupby(by=['RUID','Admission_date'])
# could also look at which service codes occurred for each visit?
# if we do this we can mostly eliminate the double admit problem
# do we also want to look at all indications/problems for a given visit?
# ANSWERED: No, SVC is probably collinear with other information we have, so let's not worry about it.

0          4.0
1          4.0
2          4.0
3          4.0
4          5.0
5          5.0
6          5.0
7          5.0
8          5.0
9          3.0
10         3.0
11         3.0
12         4.0
13         4.0
14         4.0
15         4.0
16         3.0
17         3.0
18         3.0
19         3.0
20         3.0
21         3.0
22         6.0
23         6.0
24         6.0
25         6.0
26         6.0
27         6.0
28         3.0
29         3.0
          ... 
121498     3.0
121499     6.0
121500     6.0
121501     6.0
121502     6.0
121503     6.0
121504     6.0
121505     3.0
121506     3.0
121507     3.0
121508     5.0
121509     5.0
121510     5.0
121511     5.0
121512     5.0
121513     5.0
121514     5.0
121515     5.0
121516     5.0
121517     5.0
121518    10.0
121519    10.0
121520    10.0
121521    10.0
121522    10.0
121523    10.0
121524    10.0
121525    10.0
121526    10.0
121527    10.0
Name: SRV_CODE, Length: 87076, dtype: float64

## (Attempting to) Impute missing discharge dates from CPT hospitalization & discharge codes

In [19]:
hosp_ed_cpts = ["99217", "99218", "99219", "99220", "99221", "99222", "99223", "99224", "99225", "99226", "99231", "99232", "99233", "99234", "99235", "99236", "99238", "99239", "99251", "99252", "99253", "99254", "99255", "99289","99290", "99291", "99292", "99293", "99294", "99295","99296", "99297", "99356", "99357", "99358", "99359", "99433", "99435", "99460", "99461", "99462", "99463", "99466", "99467", "99468", "99469","99471", "99472", "99475", "99476", "99477", "99478", "99479", "99480", "99485", "99486", "99281", "99282", "99283", "99284", "99285"]
cpt_pat = "|".join(hosp_ed_cpts)
disch_pat = "|".join(["99217", "99238", "99239"])

In [20]:
cpt_hosp = cpt[cpt.CPT_Code.str.match(cpt_pat)].sort_values(by=['RUID','Event_date','CPT_Code'])

In [199]:
missing_discharge = adt_cms[(adt_cms.Event == "Admit") & (adt_cms.DISCHARGE_DATE.isnull())].copy().reset_index()
missing_discharge['IMPUTED_DISCHARGE'] = missing_discharge.DISCHARGE_DATE
missing_discharge.head()

,index,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,IMPUTED_DISCHARGE
0,1757,50141473,Admit,2015-06-26,2015-06-26,GMD,DETOX,NaT,NaT
1,2000,51319626,Admit,2013-11-19,2013-11-19,ORT,716.17/996.78,NaT,NaT
2,2257,52765702,Admit,2014-12-05,2014-12-05,GMD,BACK PAIN,NaT,NaT
3,2507,53719335,Admit,2014-09-15,2014-09-15,EMR,INTRACTABLE VOMITING AND DEHYDRATION,NaT,NaT
4,2841,53725969,Admit,2011-09-27,2011-09-27,EMR,SOB,NaT,NaT


In [200]:
for idx, row in missing_discharge.iterrows():
    cpt_sub = cpt_hosp[(cpt_hosp.RUID == row.RUID) & (cpt_hosp.Event_date > row.Admission_date)]
    cpt_disch = cpt_sub[cpt_sub.CPT_Code.str.match(disch_pat)]
    orig_idx = row.index
    
    if cpt_disch.shape[0]:
        missing_discharge.IMPUTED_DISCHARGE[idx] = cpt_disch.iloc[0,2]
        # this will modify the original df
        # but we should probable be careful about that
        # so i'm commenting it out
        # adt_cms.DISCHARGE_DATE[orig_idx] = cpt_disch.iloc[0,2]

# find gaps in CPT codes -- use the last code before a non-contiguous gap and put in as discharge codes

/Users/kkondratieff/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [201]:
missing_discharge
# this isn't a reliable way of doing this -- sometimes there aren't any discharge codes for a given admit so it picks one several months later

,index,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,IMPUTED_DISCHARGE
0,1757,50141473,Admit,2015-06-26,2015-06-26,GMD,DETOX,NaT,2015-07-18
1,2000,51319626,Admit,2013-11-19,2013-11-19,ORT,716.17/996.78,NaT,NaT
2,2257,52765702,Admit,2014-12-05,2014-12-05,GMD,BACK PAIN,NaT,2014-12-08
3,2507,53719335,Admit,2014-09-15,2014-09-15,EMR,INTRACTABLE VOMITING AND DEHYDRATION,NaT,2015-01-10
4,2841,53725969,Admit,2011-09-27,2011-09-27,EMR,SOB,NaT,NaT
5,2960,53727824,Admit,2013-11-13,2013-11-13,VAS,453.40 ACUTE VENOUS EMBOLISM AND THROMBOSIS,NaT,NaT
6,4022,53727917,Admit,2012-06-17,2012-06-17,ORT,813.23 RADIUS WITH ULNA; FRACTURE OF AND ULNA ...,NaT,NaT
7,5575,53728030,Admit,2015-08-12,2015-08-12,EMR,AIDS ENTEROPATHY; FAILURE TO THRIVE; DEHYDRATION,NaT,NaT
8,6714,53728118,Admit,2015-02-03,2015-02-03,CAR,HYPERTENSIVE EMERGENCY; CHEST PAIN,NaT,NaT
9,7533,53728161,Admit,2014-10-29,2014-10-29,CAR,VOLUME OVERLOAD HEART FAILURE,NaT,2015-04-02
